# 05 - Model Analysis & Visualization

**Objective:** Understand what drives our model's predictions

**Analysis:**
1. Feature Importance (Random Forest, XGBoost, LightGBM)
2. SHAP Values (explain individual predictions)
3. Prediction Error Analysis
4. Feature Correlations
5. Geographic Patterns (if lat/lon important)

**Input:** Trained models from notebook 04

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
import json
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_absolute_error
import shap

# Plotting settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

print("Model Analysis & Visualization")

In [ ]:
# Load data and best model
ROOT = Path(r"c:\Users\lpnhu\Downloads\home-price-prediction")
DATA_DIR = ROOT / 'data'
MODELS_DIR = ROOT / 'models'
PLOTS_DIR = ROOT / 'plots'
PLOTS_DIR.mkdir(exist_ok=True)

X_train = pd.read_csv(DATA_DIR / 'X_train.csv')
X_test = pd.read_csv(DATA_DIR / 'X_test.csv')
y_train = pd.read_csv(DATA_DIR / 'y_train.csv')['ClosePrice'].values
y_test = pd.read_csv(DATA_DIR / 'y_test.csv')['ClosePrice'].values

# Load best model
best_model = joblib.load(MODELS_DIR / 'best_advanced_model.joblib')

# Load summary
with open(MODELS_DIR / 'advanced_models_summary.json') as f:
    summary = json.load(f)

print(f"Best Model: {summary['best_model']}")
print(f"Test R²: {summary['best_r2']:.4f}")
print(f"Features: {X_train.shape[1]}")
print(f"Samples: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

## 1. Feature Importance Analysis

In [ ]:
# Get feature importance
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Top 30 features
    top_30 = importance_df.head(30)
    
    plt.figure(figsize=(12, 10))
    plt.barh(range(len(top_30)), top_30['importance'], color='steelblue')
    plt.yticks(range(len(top_30)), top_30['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 30 Features - {summary["best_model"]}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'feature_importance_top30.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nTop 20 Most Important Features:")
    print(importance_df.head(20).to_string(index=False))
    
    # Save full importance
    importance_df.to_csv(MODELS_DIR / 'feature_importance.csv', index=False)
else:
    print("Model doesn't support feature_importances_")

## 2. Prediction vs Actual Analysis

In [ ]:
# Predictions
y_pred_test = best_model.predict(X_test)

# Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test, y_pred_test, alpha=0.3, s=10)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Price ($)')
axes[0].set_ylabel('Predicted Price ($)')
axes[0].set_title(f'Actual vs Predicted (R² = {summary["best_r2"]:.4f})')
axes[0].legend()

# Residuals
residuals = y_test - y_pred_test
axes[1].scatter(y_pred_test, residuals, alpha=0.3, s=10)
axes[1].axhline(0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Price ($)')
axes[1].set_ylabel('Residuals ($)')
axes[1].set_title('Residual Plot')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'prediction_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Residual statistics
print("\nResidual Statistics:")
print(f"Mean Residual: ${residuals.mean():,.0f}")
print(f"Std Residual: ${residuals.std():,.0f}")
print(f"MAE: ${mean_absolute_error(y_test, y_pred_test):,.0f}")
print(f"Median Residual: ${np.median(residuals):,.0f}")

## 3. Error Distribution Analysis

In [ ]:
# Percentage errors
pct_errors = (residuals / y_test) * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Absolute residuals histogram
axes[0].hist(residuals / 1000, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Residual ($1000s)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Residuals')
axes[0].axvline(0, color='r', linestyle='--', lw=2)

# Percentage errors
axes[1].hist(pct_errors, bins=50, edgecolor='black', alpha=0.7, range=(-50, 50))
axes[1].set_xlabel('Percentage Error (%)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Percentage Errors')
axes[1].axvline(0, color='r', linestyle='--', lw=2)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'error_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nPercentage Error Statistics:")
print(f"Mean: {pct_errors.mean():.2f}%")
print(f"Median: {np.median(pct_errors):.2f}%")
print(f"Std: {pct_errors.std():.2f}%")
print(f"Within ±10%: {(np.abs(pct_errors) <= 10).sum() / len(pct_errors) * 100:.1f}%")
print(f"Within ±20%: {(np.abs(pct_errors) <= 20).sum() / len(pct_errors) * 100:.1f}%")

## 4. Price Range Performance

In [ ]:
# Bin prices into ranges
price_bins = [0, 200000, 400000, 600000, 800000, 1000000, np.inf]
labels = ['<200K', '200-400K', '400-600K', '600-800K', '800K-1M', '>1M']

df_analysis = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred_test,
    'residual': residuals,
    'pct_error': pct_errors,
    'price_range': pd.cut(y_test, bins=price_bins, labels=labels)
})

# Performance by price range
range_stats = df_analysis.groupby('price_range').agg({
    'actual': 'count',
    'pct_error': ['mean', 'std'],
    'residual': lambda x: r2_score(df_analysis.loc[x.index, 'actual'], 
                                   df_analysis.loc[x.index, 'predicted'])
}).round(2)

range_stats.columns = ['Count', 'Mean % Error', 'Std % Error', 'R²']
print("\nPerformance by Price Range:")
print(range_stats.to_string())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean absolute % error by range
range_stats['Mean Abs % Error'] = df_analysis.groupby('price_range')['pct_error'].apply(lambda x: np.abs(x).mean())
range_stats['Mean Abs % Error'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Price Range')
axes[0].set_ylabel('Mean Absolute % Error')
axes[0].set_title('Error by Price Range')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

# R² by range
range_stats['R²'].plot(kind='bar', ax=axes[1], color='green')
axes[1].set_xlabel('Price Range')
axes[1].set_ylabel('R²')
axes[1].set_title('R² by Price Range')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)
axes[1].axhline(summary['best_r2'], color='r', linestyle='--', label='Overall R²')
axes[1].legend()

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'performance_by_price_range.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. SHAP Values (Feature Impact on Predictions)

**Note:** This can be slow for large datasets. We'll use a sample.

In [ ]:
print("Calculating SHAP values (this may take a few minutes)...\n")

# Sample for SHAP (1000 samples for speed)
sample_size = min(1000, len(X_test))
sample_idx = np.random.choice(len(X_test), sample_size, replace=False)
X_sample = X_test.iloc[sample_idx]

try:
    # Create SHAP explainer
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_sample)
    
    # Summary plot
    plt.figure()
    shap.summary_plot(shap_values, X_sample, max_display=20, show=False)
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'shap_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Feature importance (mean abs SHAP)
    shap_importance = pd.DataFrame({
        'feature': X_sample.columns,
        'mean_abs_shap': np.abs(shap_values).mean(axis=0)
    }).sort_values('mean_abs_shap', ascending=False)
    
    print("\nTop 20 Features by Mean |SHAP|:")
    print(shap_importance.head(20).to_string(index=False))
    
    shap_importance.to_csv(MODELS_DIR / 'shap_importance.csv', index=False)
    
except Exception as e:
    print(f"SHAP calculation failed: {e}")
    print("Skipping SHAP analysis (may not be supported for this model type)")

## 6. Top Feature Correlations

In [ ]:
# Get top 15 features
if 'importance_df' in locals():
    top_features = importance_df.head(15)['feature'].tolist()
    
    # Correlation matrix
    corr_matrix = X_train[top_features].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Correlation Matrix - Top 15 Features')
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'feature_correlations.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Feature importance not available")

## 7. Key Insights Summary

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)

print(f"\n1. Model Performance:")
print(f"   - Best Model: {summary['best_model']}")
print(f"   - Test R²: {summary['best_r2']:.4f}")
print(f"   - vs Steph: {summary['improvement']:+.4f}")

if 'importance_df' in locals():
    print(f"\n2. Top 5 Most Important Features:")
    for i, row in importance_df.head(5).iterrows():
        print(f"   {i+1}. {row['feature']}: {row['importance']:.4f}")

print(f"\n3. Prediction Accuracy:")
print(f"   - Mean Absolute Error: ${mean_absolute_error(y_test, y_pred_test):,.0f}")
print(f"   - Predictions within ±10%: {(np.abs(pct_errors) <= 10).sum() / len(pct_errors) * 100:.1f}%")
print(f"   - Predictions within ±20%: {(np.abs(pct_errors) <= 20).sum() / len(pct_errors) * 100:.1f}%")

print(f"\n4. Model Characteristics:")
print(f"   - Systematic bias: {residuals.mean():+,.0f} (near 0 is good)")
print(f"   - Residual spread: ${residuals.std():,.0f}")

print(f"\n5. Recommendations:")
if summary['best_r2'] > 0.884:
    print("   ✅ Model beats Steph's baseline!")
    print("   - Consider ensemble methods for further improvement")
    print("   - Analyze feature engineering opportunities")
else:
    gap = (0.884 - summary['best_r2']) * 100
    print(f"   ⚠️  Still {gap:.2f}% behind Steph")
    print("   - Try stacking/blending multiple models")
    print("   - Consider more aggressive feature engineering")
    print("   - Investigate outliers and data quality")

print("\n" + "="*80)
print(f"Plots saved to: {PLOTS_DIR}")
print(f"Analysis files saved to: {MODELS_DIR}")
print("\n✅ Analysis complete!")